In [8]:
import os
from imutils.video import VideoStream
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [9]:
DATA_DIR1 = "rockpaperscissors/"
CATEGORIES = ["paper", "rock", "scissors"]
IMG_WIDTH = 75
IMG_HEIGHT = 50
BATCH_SIZE = 150

def load_data(data_dir1, CATEGORIES, img_width, img_height):
    X = []
    y = []
    index = 0

    for category in CATEGORIES:
        one_hot = np.zeros(len(CATEGORIES))  # to encode the class as a one hot vector
        one_hot[index] = 1
        path = os.path.join(data_dir1, category)
        index += 1

        for img in os.listdir(path): # get all images in the path
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.resize(img_arr, (img_width, img_height))
            img_arr = np.asarray(img_arr)
            X.append(img_arr)
            y.append(one_hot)

    return X, y

X, y = load_data(DATA_DIR1, CATEGORIES, IMG_WIDTH, IMG_HEIGHT)

In [10]:
DATA_DIR2 = "rockpaperscissors"
CATEGORIES = ["paper", "rock", "scissors"]
IMG_WIDTH = 75
IMG_HEIGHT = 50
BATCH_SIZE = 100

def load_data(data_dir2, CATEGORIES, img_width, img_height):
    X = []
    y = []
    index = 0
    
    for category in CATEGORIES:
        one_hot = np.zeros(len(CATEGORIES))  # to encode the class as a one hot vector
        one_hot[index] = 1
        path = os.path.join(data_dir2, category)
        index += 1
        
        for img in os.listdir(path): # get all images in the path
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.resize(img_arr, (img_width, img_height))
            img_arr = np.asarray(img_arr)
            X.append(img_arr)
            y.append(one_hot)
            
    return X, y

X2, y2 = load_data(DATA_DIR2, CATEGORIES, IMG_WIDTH, IMG_HEIGHT)
X += X2
y+= y2

In [11]:
# Divide our data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

# Normalize our data
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because we are using greyscale, we only have a single channel - RGB colour images would have 3
X_train = X_train.reshape(X_train.shape[0], IMG_WIDTH, IMG_HEIGHT, 1)
X_test = X_test.reshape(X_test.shape[0], IMG_WIDTH, IMG_HEIGHT, 1)
input_shape = (IMG_WIDTH, IMG_HEIGHT, 1)

# convert the data to the right type
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# X_train /= 50
# X_test /= 50
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

pass

x_train shape: (3500, 75, 50, 1)
3500 train samples
876 test samples


In [12]:
# Build Model


# Set the model type as sequential 
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(34, kernel_size=(3, 3), strides=(1,1), activation="relu", input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(1,1)))

model.add(tf.keras.layers.Conv2D(34, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(45, activation="relu"))
model.add(tf.keras.layers.Dropout(.45))
model.add(tf.keras.layers.Dense(len(CATEGORIES), activation="softmax"))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.SGD(lr=0.02),
              metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train),
          batch_size=BATCH_SIZE,
          epochs=100,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)))

Train on 3500 samples, validate on 876 samples
Epoch 1/2
3500/3500 [==============================] - 24s 7ms/sample - loss: 1.0978 - accuracy: 0.3546 - val_loss: 1.0964 - val_accuracy: 0.4920
Epoch 2/2
3500/3500 [==============================] - 30s 8ms/sample - loss: 1.0961 - accuracy: 0.3803 - val_loss: 1.0944 - val_accuracy: 0.5491


In [ ]:
# function to parse single image
def load_image(image_path):
   img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
   img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
   img = np.asarray(img)
   #normalize image
   img = tf.keras.utils.normalize(img, axis=1)
   #reshape into tensor
   img = img.reshape(1, IMG_WIDTH, IMG_HEIGHT, 1)
   return img

In [ ]:
# test model with selected image
rock = "rockpaperscissors/rock/tGBAO9oyBYRD7XXS.png"
paper = "rockpaperscissors/paper/tiS1ECooPwR5IvFK.png"
scissors = "rockpaperscissors/scissors/pZ9zE5ahoKZhIZqm.png"
catagories = ['paper', 'rock', 'scissors']
test_img = [rock, paper, scissors]
selector = 0
prediction = model.predict_classes(load_image(test_img[selector]))
plt.imshow(cv2.imread(test_img[x]))
print("predicted action is", catagories[prediction[0]])

In [13]:
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0943992671356897
Test accuracy: 0.54908675


In [1]:

def detect_and_display(model, video_source):
    # Get frame from video soruce
    frame = video_source.read()
    
    # Convert image for model
    small = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
    gray = cv2.cvtColor(small, cv2.COLOR_RGB2GRAY)
    # Reshape for input to NN
    img = tf.keras.utils.normalize(img, axis=1)
    img_arr = gray.reshape(1, IMG_WIDTH, IMG_HEIGHT, 1)
    # Cast to float to handle error
    img_arr = tf.cast(img_arr, tf.float32)
    # Save colour image to show to user
    colour = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    prediction = model.predict(img_arr)
    # Convert prediction from one hot to category
    index = tf.argmax(prediction[0], axis=0)
    prediction = CATEGORIES[index]
    
    cv2.putText(frame, prediction, (20, 50), cv2.FONT_HERSHEY_SIMPLEX,3, (0, 255, 0), 2)
    return frame
    
    
def live_detection(model):
    video_source = VideoStream(src=0).start()
    
    while True:
        frame = detect_and_display(model, video_source)
        cv2.imshow("Face Liveness Detector", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
    video_source.stop()
    
live_detection(model)

NameError: name 'model' is not defined